<a href="https://colab.research.google.com/github/yjyjyjy/agent/blob/main/notebooks/huggingface_sentence_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.5 MB/s eta 0:00:00


In [2]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification") # uses facebook/bart-large-mnli revision c626438

# Classification of the messages:
# 1. Problems with products/services
# 2. Interested in products or services
# 3. Collaboration/partnership opportunities
# 4. Compliments
# 5. Hate Speech
# 6. Random

classifier(
    "Dear Marina,I just wanted to say a big thank you for your amazing YouTube content that has helped me improve my English so much! 🙌📚 Your videos are awesome, and I'm truly grateful for all the valuable lessons and tips you share. Keep up the fantastic work! 💓👏🌟 ",
    candidate_labels=["problem", "interest", "collboration or partnership", "compliment", "hate speech", "random"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': " Dear Marina,I just wanted to say a big thank you for your amazing YouTube content that has helped me improve my English so much! 🙌📚 Your videos are awesome, and I'm truly grateful for all the valuable lessons and tips you share. Keep up the fantastic work! 💓👏🌟 ",
 'labels': ['compliment',
  'collboration or partnership',
  'interest',
  'problem',
  'random',
  'hate speech'],
 'scores': [0.8279862403869629,
  0.0950382649898529,
  0.04847017303109169,
  0.01488949079066515,
  0.011093717999756336,
  0.002522149356082082]}